In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"#"2"

In [1]:
# import estimator class
from estimators.rnnEstimator import RnnEstimator
from estimators.kalmanEstimator import KalmanEstimator
from estimators.particleFilterEstimator import ParticleFilterEstimator

#import rewarder class
from rewarders.thresholdRewarder import ThresholdRewarder

from matplotlib import pyplot
from utils.sequences_treatment import *
from utils.agent_treatment import *
from utils.plots import *
from utils.save import *

# import functions from Keras for the RNN
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, LSTM, SimpleRNN#, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

In [2]:
# Facultatively load a workspace
workspace_path=None #'workspaces/20210723-134410' # None or workspace_path (workspaces/...)

if workspace_path is not None:
    load_workspace(workspace_path, globals())
    print('Wokspace loaded.')
else:
    print('No workspace loaded.')

No workspace loaded.


In [3]:
# Set parameters
estimatorType='particle' # kalman, particle or rnn
pfType = "tumour"
seeAction=True
seeMeasurement=False
seeEstimate=False
seeTime=True
seeSumAction = True

T=12+1

threshold=4
windowSize=T


if estimatorType=='kalman':
    cost=50 # tumor_benchmark: 0 < cost=? < ?   classic_benchmark: 0 < cost=50 < 100
elif estimatorType=='particle':
    if seeMeasurement:
        cost=1000 
    else:
        cost=1000 # 0 < cost=? < 1000
elif estimatorType=='rnn':
    cost=500
print("cost=",cost)

cost= 1000


In [4]:
if estimatorType=='rnn' and 'model' not in dir(): # if a RNN model has not been loaded
    # construct and train a Sequential RNN model with keras
    numberSamples_trainRNN=500
    T_trainRNN=T
    generatorType='linear'#'linear'

    # generate sequences for training
    (objectives_trainRNN,measurements_trainRNN)=generateSequence(T_trainRNN,numberSamples=numberSamples_trainRNN,generatorType=generatorType)
    sigmas_trainRNN=randomSigma(T_trainRNN,numberSamples=numberSamples_trainRNN,p0=1-threshold/windowSize)
    measurements_corrupted_trainRNN=corruptSequence_outOfRange(measurements_trainRNN,sigmas_trainRNN)
    
    # new
    inputRNN=np.concatenate((np.expand_dims(sigmas_trainRNN,2),measurements_corrupted_trainRNN),axis=2)
    
    n_dim_meas=np.shape(measurements_corrupted_trainRNN)[2]
    n_dim_obj=np.shape(objectives_trainRNN)[2]

    model=Sequential()
    model.add(LSTM(20,input_shape=(None,n_dim_meas+1),return_sequences=True))
    model.add(LSTM(20,return_sequences=True))
    model.add(Dense(n_dim_obj,activation=None))

    model.compile(optimizer='adam',loss='mean_squared_error')
    model.summary()

    # train the RNN
    n_epochs_RNN=20
    history=model.fit(x=inputRNN,y=objectives_trainRNN,batch_size=1,epochs=n_epochs_RNN,validation_split=0.2,verbose=2)

    # plot loss
    plotRNNresults(history)
    
    idx_sample=0
    estimates_trainRNN=model.predict(inputRNN)
    plotExperiment(objectives_trainRNN, estimates_trainRNN, sigmas_trainRNN,idx_sample=idx_sample)
print('Done')

Done


In [5]:
# construct estimator
if estimatorType=='rnn':
    estimator=RnnEstimator(T,windowSize,threshold,model,generatorType,seeAction=seeAction,seeMeasurement=seeMeasurement,seeEstimate=seeEstimate,seeTime=seeTime,seeSumAction=seeSumAction)
elif estimatorType=='kalman':
    estimator=KalmanEstimator(T,windowSize,threshold,seeAction=seeAction,seeMeasurement=seeMeasurement,seeEstimate=seeEstimate,seeTime=seeTime,seeSumAction=seeSumAction)
elif estimatorType == 'particle':
    estimator=ParticleFilterEstimator(T,windowSize,threshold,generatorType=pfType,seeAction=seeAction,seeMeasurement=seeMeasurement,seeEstimate=seeEstimate,seeTime=seeTime,seeSumAction=seeSumAction)
else:
    print('ERROR: no valid estimatorType')
    
estimator.summarize()

<class 'utils.pfilter.ParticleFilter'>
Particle filter estimator
  observationsDimensions: [(12,), (1,), (1,)]
  seeAction= True
  seeMeasurement= False
  seeEstimate= False
  seeTime= True
  seeSumAction= True


In [6]:
# contruct rewarder
rewarder=ThresholdRewarder(threshold=threshold, cost=cost, windowSize=windowSize)
rewarder.summarize()

Threshold rewarder
  window size: 13
  threshold: 4
  cost: 1000
  number of measures in the window: 0


In [7]:
# generate sequences for training and validating the agent
numberSamples_train=50
T_train=T

numberSamples_valid=numberSamples_train
T_valid=T_train

if workspace_path is None: # data have not been loaded
    (objectives_train,measurements_train)=estimator.generateSequence(T_train,numberSamples=numberSamples_train)
    (objectives_valid,measurements_valid)=estimator.generateSequence(T_valid,numberSamples=numberSamples_valid)
    print('New data generated')
else:
    print('No new data generated')

#print('----- TO REMOVE -----')
#objectives_valid = objectives_train
#measurements_valid = measurements_train

print('shape training objectives:',np.shape(objectives_train))
print('shape training measurements:',np.shape(measurements_train))
print('shape validating objectives:',np.shape(objectives_valid))
print('shape validating measurements:',np.shape(measurements_valid))

New data generated
shape training objectives: (50, 13, 1)
shape training measurements: (50, 13, 1)
shape validating objectives: (50, 13, 1)
shape validating measurements: (50, 13, 1)


In [9]:
# construct agent
agent=constructAgent(estimator,rewarder,objectives_train,measurements_train,objectives_valid,measurements_valid)

agent._learning_algo.q_vals.summary()

print('Agent constructed')

Environment parameters
  inputDimensions= [(12,), (1,), (1,)]
Sequences parameters
  outOfRangeValue= -1
  n_dim_obj= 1
  n_dim_meas= 1
  numberSamples_train 50
  numberSamples_valid 50
Particle filter estimator
  observationsDimensions: [(12,), (1,), (1,)]
  seeAction= True
  seeMeasurement= False
  seeEstimate= False
  seeTime= True
  seeSumAction= True
Threshold rewarder
  window size: 13
  threshold: 4
  cost: 1000
  number of measures in the window: 0
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 12, 1)     0           input_1[0][0]                    
______________________

In [ ]:
# Train agent
n_epochs_agent=5

agent.resumeTrainingMode() # not required

start_time = time.time()
agent.run(n_epochs=n_epochs_agent, epoch_length=numberSamples_train*T_train)
elapsed_time = time.time() - start_time

cumulatedRewards_valid=agent._controllers[-1].rewards # interleavedValidEpochController is the last controller.
print('Agent Trained (in',elapsed_time,'seconds)')

C:\Users\nathan\OneDrive\UCL\Passerelle\TFE\github\optiMeasRL\utils\pfilter.py:372: RuntimeWarning: Degrees of freedom <= 0 for slice
  self.cov_state = np.cov(self.particles, rowvar=False, aweights=self.weights)
C:\Users\nathan\Anaconda3\envs\newenv\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\nathan\Anaconda3\envs\newenv\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [ ]:
# plot validation
#boxplotCumulatedRewards(cumulatedRewards_valid)
plotAllCumulatedRewards(cumulatedRewards_valid)
allHistoCumulatedRewards(cumulatedRewards_valid)

meanReward_valid=np.mean(cumulatedRewards_valid[-1])
print('meanReward_valid: ',meanReward_valid)

In [ ]:
# generate test data
numberSamples_test=numberSamples_train
T_test=T
    
if workspace_path is None: # data have not been loaded
    (objectives_test,measurements_test)=estimator.generateSequence(T_test,numberSamples=numberSamples_test)
    print('New data generated')
else:
    print('No new data generated')

#print('----- TO REMOVE -----')
#objectives_test = objectives_train
#measurements_test = measurements_train


# Results of the inference on test data
(sigmas_test,rewards_test,estimates_test)=agentInference(agent,objectives_test,measurements_test)
print('Test results computed')

In [ ]:
# plot test
idx_sample=0
#plotExperiment(objectives_test,estimates_test,sigmas_test,idx_sample=idx_sample)
freqSigmas(sigmas_test)
boxplotErrors(objectives_test,estimates_test)
#plotAllErrors(objectives_test,estimates_test)
boxplotRewards(rewards_test)
#plotAllRewards(rewards_test)
histoCumulatedRewards(rewards_test)

meanReward_test=np.mean(rewards_test)
print('meanReward_test: ',meanReward_test)

sumSigmas_test=np.sum(sigmas_test,axis=1)
print('sumSigmas_test',sumSigmas_test)

In [ ]:
# regular run on test data (default case without agent)
numberMeasurements=int(np.ceil(threshold/windowSize*T))

# compute regular sigma (with same dim than validation data)
sigmas_regular=regularSigma(T_valid,numberMeasurements,numberSamples=numberSamples_test)

(rewards_regular, estimates_regular)=agentForcedInference(agent, sigmas_regular)
print('Done.')

In [ ]:
# plot test regular
idx_sample=5
#plotExperiment(objectives_test,estimates_test,sigmas_test,idx_sample=idx_sample)
freqSigmas(sigmas_regular)
boxplotErrors(objectives_test,estimates_regular)
#plotAllErrors(objectives_test,estimates_regular)
boxplotRewards(rewards_regular)
#plotAllRewards(rewards_regular)
histoCumulatedRewards(rewards_regular)
meanReward_regular=np.mean(rewards_regular)
print('meanReward_regular: ',meanReward_regular)

In [ ]:
toSave=["estimatorType","seeAction","seeMeasurement","seeEstimate","seeTime","T","threshold","windowSize","cost"]
if estimatorType=='rnn':
    toSave.extend(["numberSamples_trainRNN","T_trainRNN","generatorType","objectives_trainRNN","measurements_trainRNN","n_epochs_RNN","model","history","estimates_trainRNN"])
toSave.extend(["estimator","rewarder"])
toSave.extend(["numberSamples_train","T_train","numberSamples_valid","T_valid","objectives_train","measurements_train","objectives_valid","measurements_valid"])
toSave.extend(["agent","n_epochs_agent"])
toSave.extend(["cumulatedRewards_valid","meanReward_valid"])
toSave.extend(["numberSamples_test","T_test","objectives_test","measurements_test","sigmas_test","rewards_test","estimates_test"])
toSave.extend(["meanReward_test","sumSigmas_test"])
toSave.extend(["numberMeasurements","sigmas_regular","rewards_regular","estimates_regular","meanReward_regular"])
save_workspace('workspaces/',toSave,globals())

In [ ]:
import os
os.system("say Au travail faignant.")

In [ ]:
agent.discountFactor()

In [ ]:
agent._learning_algo.__dict__

In [ ]:
initial_state_mean = np.array([0, 1])
n_state = len(initial_state_mean)
initial_state_covariance = np.array([[1,0],[0,1]])
n = 100

test = np.random.multivariate_normal(initial_state_mean,
                                             initial_state_covariance,
                                             n)
print(test.shape)
test2 = np.random.multivariate_normal(initial_state_mean,
                                             initial_state_covariance,
                                             test.shape[0])
print(test2.shape)

In [ ]:
np.dot(test, initial_state_covariance).shape

In [ ]:
initial_state_mean = np.array([0, 1])
n_state = len(initial_state_mean)
initial_state_covariance = np.array([[1,0],[0,1]])
delta=0.5
transition_matrix = np.array([[np.cos(delta), np.sin(delta)], [-np.sin(delta), np.cos(delta)]])
observation_matrix = np.array([1,0])
objective_matrix = np.array([[1,0]]) # Added by A. Aspeel
transition_covariance = np.array([[1/80*( delta-np.sin(delta)*np.cos(delta) ) ,
                                    1/80*np.sin(delta)**2] , [-1/80*np.sin(delta)**2 ,
                                                              1/80*( delta+np.sin(delta)*np.cos(delta) )]])
observation_covariance = np.array([[1]]) #[[1,0],[0,1]]
obs_dim = 1
#dynamic where test is x
print(np.dot(test,transition_matrix).shape)

#noise fn
noise_test = test + np.random.multivariate_normal( np.zeros(n_state), transition_covariance, test.shape[0])
print(noise_test.shape)

#observe_noise
test3 = test + np.random.normal(0, observation_covariance)
print(test3.shape)

#observe
print(np.dot(test, observation_matrix).shape)

In [ ]:
np.dot(test,transition_matrix.T).shape

In [ ]:
x shape observe: (50, 2)
x shape obs noise: (50, 1)
x shape dyn: (50, 2)
    
x shape dyn noise: (50, 2)
x shape obs noise: (50, 1)
x shape observe: (50, 2)
x shape obs noise: (50, 1)
    
hypotheses shape: (50, 1)
weights shape: (50,)


In [ ]:
x shape observe: (100, 3)
x shape obs noise: (100,)
x shape dyn : (100, 3)
    
hypotheses shape: (50, 1)
weights shape: (50,)

In [ ]:
zero = np.zeros(3)
print(zero.shape)

In [ ]:
print(test.shape)
print(np.random.normal(0, 1, obs_dim).shape)
print((zero + np.random.normal(0, 1, obs_dim)).reshape((zero.shape[0],obs_dim)).shape)

In [ ]:
t1=np.random.normal(0,1,(3,))
t2=np.random.normal(0,1,(3,3))

In [ ]:
np.dot(t1,t2).shape

In [ ]:
np.matmul(t1,t2).shape

In [16]:
(z,y,x)=samplePFSequence(estimator._pf,10,numberSamples=2)

In [14]:
z

array([[[  3.67007022],
        [  8.62270499],
        [ 12.35596318],
        [ 12.52928051],
        [  9.29377842],
        [  6.54914467],
        [  1.93430749],
        [ -3.11156152],
        [ -7.8123712 ],
        [-10.63887471]],

       [[  0.61714261],
        [  6.16445333],
        [ 10.54145115],
        [ 11.56634016],
        [  8.81384273],
        [  5.8055054 ],
        [  2.05226504],
        [ -2.74039769],
        [ -8.42626563],
        [-13.5351246 ]],

       [[ -5.241902  ],
        [  1.71599552],
        [  7.53026551],
        [ 11.15168186],
        [ 10.49012887],
        [  8.43238761],
        [  5.36628088],
        [  0.48426231],
        [ -6.81043859],
        [-13.68545004]],

       [[  3.03004796],
        [ 13.17356748],
        [ 21.23193167],
        [ 24.82602942],
        [ 22.95132168],
        [ 18.74063707],
        [ 12.27538684],
        [  3.52584684],
        [ -7.01614663],
        [-17.38031716]],

       [[ -1.04794514],
        